In [1]:
#import pandas as pd
#from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

#df_2020_2 = pd.DataFrame(client.get("rnvb-vnyh", limit=504872, offset=0))

#df_2020_1 = pd.DataFrame(client.get("a8xr-en99", limit=15435, offset=0))

In [2]:
import polars as pl
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
# define functions
def lower_names(df):
    l_names = {col: col.lower() for col in df.columns}
    return df.rename(l_names)

def plot_null_vals(df):
    null_bool = df.select([pl.col(col).is_null() for col in df.columns])
    
    null_array = null_bool.to_numpy().astype(int).T
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.imshow(null_array, aspect='auto', cmap='viridis', interpolation='none')
    
    ax.set_yticks(range(len(df.columns)))
    ax.set_yticklabels(df.columns)
    ax.set_xticks([])
    plt.title('Valores nulos por columna')
    
    return plt.show()

def data_type(df):
    df_type = pl.DataFrame(
        {
            "col_name": df.columns,
            "var_type": [df_h[col].dtype for col in df.columns]
        }
    )

    return df_type

In [4]:
# load data
df_h = pl.read_csv("/Users/sebas/OneDrive/Documents/repositorio_cepap/CEPAP/data_source/educacion/2025/Resultados__nicos_Saber_11_20250303.tsv", separator="\t", decimal_comma = True)

df_h.head()

FileNotFoundError: El sistema no puede encontrar el archivo especificado. (os error 2): Resultados__nicos_Saber_11_20250303.tsv

In [ ]:
# clean names
df_h = lower_names(df_h)

df_h.head()

In [ ]:
df_h.group_by("periodo").len()

In [ ]:
# filter periods
df_h = df_h.filter(df_h["periodo"].is_in([20171, 20172, 20191, 20194]))

df_h.shape

In [ ]:
# sample 500.000 records
df_h = df_h.sample(n = 500000, seed = 2025)

In [ ]:
#clean data type
df_h = df_h.with_columns(
    pl.col("punt_matematicas").cast(pl.Float64),
    pl.col("punt_ingles").cast(pl.Float64)
)

In [ ]:
# print data type
data_type(df_h)

In [ ]:
cols_punt = [col for col in df_h.columns if col.startswith("punt_")]

df_h.select(cols_punt).describe()

In [ ]:
# Arbitrary select vars
df_h = df_h.select(["estu_genero", "cole_calendario", "fami_educacionmadre", "fami_educacionpadre",
                    "fami_estratovivienda", "fami_tienecomputador", "cole_area_ubicacion", "fami_tieneinternet", "punt_global"])

In [ ]:
# NA values
pl.DataFrame(
    {
        "col_name": df_h.columns,
        "missing_values": [df_h[col].null_count() for col in df_h.columns]
    }
) \
.sort("missing_values", descending = True)

In [ ]:
plot_null_vals(df_h)

In [ ]:
# delete null values
df_h = df_h.drop_nulls()

In [ ]:
# verfy delete null values
plot_null_vals(df_h)

In [ ]:
print("Luego de la selección arbitraria de variables y de la eliminación de datos perdidos, se observa un total de " + str(df_h.shape[0]) +
      " filas y " + str(df_h.shape[1]) + " columnas")

In [ ]:
# print data type
data_type(df_h)

In [ ]:
# plot count unique values
var_names_plot = df_h.columns

var_names_plot.remove("punt_global")

for i in var_names_plot:
    count_val = df_h[i].value_counts()

    plt.barh(count_val[i], count_val["count"])
    plt.xlabel("Frecuencia")
    plt.ylabel(i)
    plt.show()

plt.boxplot(df_h["punt_global"])
plt.show()
